In [7]:
import pandas as pd  
import pickle
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sstats
import scipy
from scipy import signal
from scipy.integrate import simps
from scipy.signal import filtfilt, butter, lfilter, welch
import nolds
import antropy as ant
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
#%%
#LOADING DATA(s01)
with open("s32.dat", "rb") as file: f = pickle.load(file, encoding='latin1')
Ntrial = 40
data = f['data']
labels = f['labels']
CAR_all = []
Feature_trial = []
BP_Power = []
Feature_array = []
FAA_trial = []
#%%
#LOOP FOR TRIAL
for k in range(Ntrial):
    trial = data[k]
    Ctrial = trial - np.mean(trial, axis=0) #Common Average Referencing
    C_EEG_Trial =  Ctrial [0:32, 384:8064] #Seperating EEG and non-EEG channels/ 3sec baseline removal
    C_Baseline_EEG = Ctrial [0:32, 0:384]
    CAR_all.append(C_EEG_Trial)
    bp_theta_Value, bp_alpha_Value, bp_beta_Value , bp_gamma_Value  = [] , [] , [] , []
    Mean_Value, Stdev_Value, Skew_Value, Median_Value, Kurt_Value, Ent_Value, HFD_Value, Corr_Value = [] , [] , [] , [] , [], [], [], []
#%%
    #LOOP FOR EEG CHANNELS
    for i in range (np.shape(C_EEG_Trial[:,1])[0]):
    # Filtering
      order = 5
      sample_freq = 128
      cutoff_freq = 2
      sample_duration = 60
      no_of_samples = sample_freq * sample_duration
      time = np.linspace(0, sample_duration, no_of_samples, endpoint = False)
      normalized_cutoff = 2 * cutoff_freq / sample_freq
      b, a = scipy.signal.butter(order, normalized_cutoff, analog=False)
      filtered_signal = scipy.signal.lfilter(b, a, C_EEG_Trial[0:32,:], axis = 0)
      filt_window = np.hsplit(filtered_signal, 60)      
      
      #Compute PSD
      sf = 128
      time = np.arange(C_EEG_Trial.size)/ sf  
      win = 1 * sf #Define window lenght(1 second)
      freqs_filt, psd_filt = freqs, Psd = scipy.signal.welch(filtered_signal, fs=128.0, nperseg=win, axis=1)
      freq_filt_res = freqs[1] - freqs[0]
      
      #BANDPASS FILTER
      nyq = 0.5 * sample_freq
      
      #FOR THETA
      fmin_theta = 3
      fmax_theta = 7
      low_theta = fmin_theta / nyq
      high_theta = fmax_theta / nyq
      b_theta, a_theta = scipy.signal.butter(order, [low_theta, high_theta], btype='band', analog=False)
      filtered_theta = scipy.signal.lfilter(b_theta, a_theta, filtered_signal[0:32,:], axis = 0)
      filt_theta = np.hsplit(filtered_theta, 60)
     
      
      #FOR ALPHA
      fmin_alpha = 8
      fmax_alpha = 13
      low_alpha = fmin_alpha / nyq
      high_alpha = fmax_alpha / nyq
      b_alpha, a_alpha = scipy.signal.butter(order, [low_alpha, high_alpha], btype='band', analog=False)
      filtered_alpha = scipy.signal.lfilter(b_alpha, a_alpha, filtered_signal[0:32,:], axis = 0)
      filt_alpha = np.hsplit(filtered_alpha, 60)
      
      
      #FOR BETA
      fmin_beta = 14
      fmax_beta = 29
      low_beta = fmin_beta / nyq
      high_beta = fmax_beta / nyq
      b_beta, a_beta = scipy.signal.butter(order, [low_beta, high_beta], btype='band', analog='False')
      filtered_beta = scipy.signal.lfilter(b_beta, a_beta, C_EEG_Trial[0:32,:], axis = 0)
      filt_beta = np.hsplit(filtered_beta, 60)
      #FOR Frontal assymetery of beta band
      freqs_b, Psd_b = scipy.signal.welch(filtered_beta, nperseg=win, axis=1)
      idx_b = np.logical_and(freqs_filt >= fmin_beta, freqs_filt <= fmax_beta)
      pow_rightbeta = simps(Psd_b[19,:][idx_b], dx=freq_filt_res)
      pow_leftbeta = simps(Psd_b[2,:][idx_b], dx=freq_filt_res)
      #Computing FAA at F3 & F4 channel
      FAA_beta = np.log(pow_rightbeta) - np.log(pow_leftbeta)

      #FOR GAMMA
      fmin_gamma = 30
      fmax_gamma = 47
      low_gamma = fmin_gamma / nyq
      high_gamma = fmax_gamma / nyq
      b_gamma, a_gamma = scipy.signal.butter(order, [low_gamma, high_gamma], btype='band', analog='False')
      filtered_gamma = scipy.signal.lfilter(b_gamma, a_gamma, C_EEG_Trial[0:32,:], axis = 0)
      filt_gamma = np.hsplit(filtered_gamma, 60)
#%%
      #Loop for 60 chunks
      #Compute PSD, bandpower
      for j in range (60):
          freqs, Psd = scipy.signal.welch(((filt_window[j])[i]),fs=128, nperseg = win, axis=0 )
          freqs_theta, Psd_theta = scipy.signal.welch(((filt_theta[j])[i]), fs=128, nperseg = win, axis=0)
          freqs_alpha, Psd_alpha = scipy.signal.welch(((filt_alpha[j])[i]), fs=128, nperseg = win, axis=0)
          freqs_beta, Psd_beta = scipy.signal.welch(((filt_beta[j])[i]), fs=128, nperseg = win, axis=0)
          freqs_gamma, Psd_gamma = scipy.signal.welch(((filt_gamma[j])[i]), fs=128, nperseg = win, axis=0)
          freqs_res = freqs[1] - freqs[0]
          idx_theta = np.logical_and(freqs >= fmin_theta, freqs <= fmax_theta)
          idx_alpha = np.logical_and(freqs >= fmin_alpha, freqs <= fmax_alpha)
          idx_beta = np.logical_and(freqs >= fmin_beta, freqs <= fmax_beta)
          idx_gamma = np.logical_and(freqs >= fmin_gamma, freqs <= fmax_gamma)
          bp_theta = simps(Psd_theta[idx_theta], dx=freqs_res)
          bp_alpha = simps(Psd_alpha[idx_alpha], dx=freqs_res)
          bp_beta = simps(Psd_beta[idx_beta], dx=freqs_res)
          bp_gamma = simps(Psd_gamma[idx_gamma], dx=freqs_res)
          bp_theta_Value.append(bp_theta)
          bp_alpha_Value.append(bp_alpha)
          bp_beta_Value.append(bp_beta)
          bp_gamma_Value.append(bp_gamma)
          
    n = len(filt_window)
    for l in range (n):
        for m in range (len(filt_window[l])):
            w_mean = np.mean((filt_window[l])[m])
            w_stdev = np.std((filt_window[l])[m])
            w_skew = sstats.skew((filt_window[l])[m])
            w_median = np.median((filt_window[l])[m])
            w_kurt = sstats.kurtosis((filt_window[l])[m])
            ent = ant.sample_entropy((filt_window[l])[m])
            hfd = ant.higuchi_fd((filt_window[l])[m])
            corr_dim = nolds.corr_dim(((filt_window[l])[m]), emb_dim=2)
            Mean_Value.append(w_mean)
            Stdev_Value.append(w_stdev)
            Skew_Value.append(w_skew)
            Median_Value.append(w_median)
            Kurt_Value.append(w_kurt)
            Ent_Value.append(ent)
            HFD_Value.append(hfd)
            Corr_Value.append(corr_dim)
    FAA_trial.append(FAA_beta)
    frontal_assym = np.array(FAA_trial)
    Feature_array.append([Skew_Value, Kurt_Value, Ent_Value, HFD_Value, Corr_Value, bp_gamma_Value, bp_theta_Value, bp_alpha_Value, bp_beta_Value])
    Feature = np.array(Feature_array)
    Feature_reshape = np.reshape(Feature, (Feature.shape[0], Feature.shape[1]*Feature.shape[2]))
    Feature_matrix = np.column_stack((Feature_reshape, frontal_assym))
    df_Feature = pd.DataFrame(Feature_matrix)
    df_Feature[df_Feature==np.inf]=np.nan
    df_Feature.fillna(df_Feature.mean(), inplace=True)
    scaler=StandardScaler()
    Feature_rescaled = scaler.fit_transform(Feature_matrix)
    pca = PCA(n_components = 0.79)
    pca.fit(Feature_rescaled)
    Feature_PCA = pca.transform(Feature_rescaled)

C:\Users\khush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\decomposition\_pca.py:541: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)


In [8]:
Feature_PCA.shape

(40, 24)

In [9]:
df_Feature.head(5)

,0,1,2,3,4,5,6,7,8,9,...,17271,17272,17273,17274,17275,17276,17277,17278,17279,17280
0,0.849016,0.840435,0.818074,0.784363,0.742739,0.696797,0.649754,0.603844,0.559939,0.517969,...,0.000007,0.000009,0.000011,0.000012,0.000006,0.000007,0.000008,0.000010,0.000012,-0.693338
1,-0.735266,-0.761742,-0.720669,-0.695739,-0.716319,-0.770639,-0.835874,-0.896357,-0.946264,-0.985525,...,0.000004,0.000010,0.000009,0.000006,0.000008,0.000012,0.000009,0.000005,0.000009,0.651083
2,-0.269204,-0.033480,0.151840,0.147768,0.015213,-0.140698,-0.264054,-0.343453,-0.386543,-0.404026,...,0.000007,0.000007,0.000010,0.000012,0.000003,0.000004,0.000008,0.000007,0.000006,0.319916
3,-0.216745,-0.321076,-0.420056,-0.484279,-0.500206,-0.470094,-0.405907,-0.322724,-0.233726,-0.148026,...,0.000008,0.000004,0.000008,0.000004,0.000007,0.000004,0.000003,0.000006,0.000004,-0.171926
4,-0.347387,-0.301131,-0.318010,-0.365077,-0.415158,-0.459538,-0.496075,-0.525158,-0.548217,-0.566795,...,0.000022,0.000008,0.000002,0.000012,0.000017,0.000023,0.000018,0.000020,0.000010,0.429005


In [10]:
df_Feature.shape

(40, 17281)

In [11]:
Feature.shape

(40, 9, 1920)